In [3]:
import pandas as pd
import numpy as np
import math
import unicodedata
import re
from sklearn.preprocessing import MinMaxScaler
from datasets import load_dataset, concatenate_datasets
import json

In [4]:
df_kt = pd.read_json(r"C:\Users\asus\Desktop\ACM\data_kt\problems_data.jsonl", lines = True)

df_kt.info()
print()
print(df_kt["problem_score"].min(), df_kt["problem_score"].max())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4112 entries, 0 to 4111
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               4112 non-null   object 
 1   description         4112 non-null   object 
 2   input_description   4112 non-null   object 
 3   output_description  4112 non-null   object 
 4   sample_io           4112 non-null   object 
 5   problem_class       4112 non-null   object 
 6   problem_score       4112 non-null   float64
 7   url                 4112 non-null   object 
dtypes: float64(1), object(7)
memory usage: 257.1+ KB

1.1 9.7


In [ ]:
dfc = df_kt.sort_values("problem_score").reset_index(drop=True)

easy = dfc[dfc["problem_class"] == "easy"]["problem_score"]
medium = dfc[dfc["problem_class"] == "medium"]["problem_score"]
hard = dfc[dfc["problem_class"] == "hard"]["problem_score"]

print(medium.min())
print(hard.min())

#i have used this info later to create problem_class column in df_cf

2.8
5.5


In [6]:
def sample_io_to_text(sample):
    if not isinstance(sample, list):
        return ""

    parts = []
    for i, s in enumerate(sample, 1):
        if isinstance(s, dict):
            inp = s.get("input", "")
            out = s.get("output", "")
            parts.append(f"Input: {inp} Output: {out}")
        else:
            parts.append(str(s))
    return " ".join(parts)

df_kt["sample_io_text"] = df_kt["sample_io"].apply(sample_io_to_text)

In [7]:
df_kt["score_norm"] = MinMaxScaler().fit_transform( df_kt[["problem_score"]] )

In [8]:
df_kt = df_kt.drop(columns=["url"])

In [ ]:
text_cols = ["title", "description", "input_description", "output_description", "sample_io_text"]

def clean_text(text):
    text = text.lower()
    text = re.sub(r"\s+", " ", text)
    return text.strip()

for col in text_cols:
    df_kt[col] = df_kt[col].apply(clean_text)

In [10]:
df_kt["combined_text"] = ("Title: " + df_kt["title"] + " "
    "Description: " + df_kt["description"] + " "
    "Input: " + df_kt["input_description"] + " "
    "Output: " + df_kt["output_description"] + " "
    "Sample IO: " + df_kt["sample_io_text"]
)
df_kt["combined_text"] = df_kt["combined_text"].apply(clean_text)

In [35]:
dt = load_dataset(
    "parquet",
    data_files={
        "train": r"C:/Users/asus/Desktop/ACM/data_cf/train-*.parquet",
        "test": r"C:/Users/asus/Desktop/ACM/data_cf/test-*.parquet",
    }
)

print(dt.column_names)

req_cols = ["title", "description", "input_format", "output_format", "examples", "rating"]
rem_cols = []

for col in dt["train"].column_names:
    if col not in req_cols:
        rem_cols.append(col)

dt["train"] = dt["train"].remove_columns(rem_cols)
dt["test"]  = dt["test"].remove_columns(rem_cols)

{'train': ['id', 'aliases', 'contest_id', 'contest_name', 'contest_type', 'contest_start', 'contest_start_year', 'index', 'time_limit', 'memory_limit', 'title', 'description', 'input_format', 'output_format', 'interaction_format', 'note', 'examples', 'editorial', 'rating', 'tags', 'testset_size', 'official_tests', 'official_tests_complete', 'input_mode', 'generated_checker', 'executable', 'generated_tests'], 'test': ['id', 'aliases', 'contest_id', 'contest_name', 'contest_type', 'contest_start', 'contest_start_year', 'index', 'time_limit', 'memory_limit', 'title', 'description', 'input_format', 'output_format', 'interaction_format', 'note', 'examples', 'editorial', 'rating', 'tags', 'testset_size', 'official_tests', 'official_tests_complete', 'input_mode', 'generated_checker', 'executable', 'generated_tests']}


In [12]:
dt_c = concatenate_datasets([dt["train"], dt["test"]])

df_cf = dt_c.to_pandas()
df_cf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10024 entries, 0 to 10023
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          10024 non-null  object 
 1   description    9981 non-null   object 
 2   input_format   9862 non-null   object 
 3   output_format  9742 non-null   object 
 4   examples       9895 non-null   object 
 5   rating         9767 non-null   float64
dtypes: float64(1), object(5)
memory usage: 470.0+ KB


In [13]:
text_cols = ["description", "input_format", "output_format", "examples"]
df_cf[text_cols] = df_cf[text_cols].fillna("")

df_cf = df_cf.dropna(subset=["rating"])


In [14]:
df_cf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9767 entries, 0 to 10023
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          9767 non-null   object 
 1   description    9767 non-null   object 
 2   input_format   9767 non-null   object 
 3   output_format  9767 non-null   object 
 4   examples       9767 non-null   object 
 5   rating         9767 non-null   float64
dtypes: float64(1), object(5)
memory usage: 534.1+ KB


In [15]:
df_cf["rating"].describe()

count    9767.000000
mean     1857.090202
std       721.583414
min       800.000000
25%      1300.000000
50%      1800.000000
75%      2400.000000
max      3500.000000
Name: rating, dtype: float64

In [16]:
df_cf["score_norm"] = MinMaxScaler().fit_transform( df_cf[["rating"]] )

In [17]:

def score_to_class(score):
    if score < 0.28:
        return "easy"
    elif score < 0.55:
        return "medium"
    else:
        return "hard"

df_cf["problem_class"] = df_cf["score_norm"].apply(score_to_class)


In [18]:
def examples_to_text(examples):
    if examples is None or (isinstance(examples, float) and np.isnan(examples)):
        return ""

    blocks = []

    for ex in list(examples):
        if not isinstance(ex, dict):
            continue

        inp = ex.get("input", "")
        out = ex.get("output", "")
        blocks.append(f"Input:\n{inp}\nOutput:\n{out}")

    return "\n\n".join(blocks)

df_cf["sample_io_text"] = df_cf["examples"].apply(examples_to_text)

df_cf = df_cf.rename(columns={
    "input_format": "input_description",
    "output_format": "output_description",
    "rating": "problem_score",
    "examples" : "sample_io"
})

df_cf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9767 entries, 0 to 10023
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               9767 non-null   object 
 1   description         9767 non-null   object 
 2   input_description   9767 non-null   object 
 3   output_description  9767 non-null   object 
 4   sample_io           9767 non-null   object 
 5   problem_score       9767 non-null   float64
 6   score_norm          9767 non-null   float64
 7   problem_class       9767 non-null   object 
 8   sample_io_text      9767 non-null   object 
dtypes: float64(2), object(7)
memory usage: 763.0+ KB


In [19]:
df_cf["combined_text"] = ("Title: " + df_cf["title"] + " "
    "Description: " + df_cf["description"] + " "
    "Input: " + df_cf["input_description"] + " "
    "Output: " + df_cf["output_description"] + " "
    "Sample IO: " + df_cf["sample_io_text"]
)
df_cf["combined_text"] = df_cf["combined_text"].apply(clean_text)

In [20]:
df_cf = df_cf[df_kt.columns]

In [21]:
df_cf.info()
print()
df_kt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9767 entries, 0 to 10023
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               9767 non-null   object 
 1   description         9767 non-null   object 
 2   input_description   9767 non-null   object 
 3   output_description  9767 non-null   object 
 4   sample_io           9767 non-null   object 
 5   problem_class       9767 non-null   object 
 6   problem_score       9767 non-null   float64
 7   sample_io_text      9767 non-null   object 
 8   score_norm          9767 non-null   float64
 9   combined_text       9767 non-null   object 
dtypes: float64(2), object(8)
memory usage: 839.4+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4112 entries, 0 to 4111
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               4112 non-null   object 


In [22]:
df = pd.concat([df_cf, df_kt], axis=0, ignore_index=True  )
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13879 entries, 0 to 13878
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               13879 non-null  object 
 1   description         13879 non-null  object 
 2   input_description   13879 non-null  object 
 3   output_description  13879 non-null  object 
 4   sample_io           13879 non-null  object 
 5   problem_class       13879 non-null  object 
 6   problem_score       13879 non-null  float64
 7   sample_io_text      13879 non-null  object 
 8   score_norm          13879 non-null  float64
 9   combined_text       13879 non-null  object 
dtypes: float64(2), object(8)
memory usage: 1.1+ MB


In [23]:
print(df.loc[166, "combined_text"])
print()
print(df.loc[1111, "combined_text"])
print()
print(df.loc[4567, "combined_text"])
print()
print(df.loc[12345, "combined_text"])  

Title: Game with Multiset Description: In this problem, you are initially given an empty multiset. You have to process two types of queries: 1. ADD $$$x$$$ — add an element equal to $$$2^{x}$$$ to the multiset; 2. GET $$$w$$$ — say whether it is possible to take the sum of some subset of the current multiset and get a value equal to $$$w$$$. Input: The first line contains one integer $$$m$$$ ($$$1 \le m \le 10^5$$$) — the number of queries. Then $$$m$$$ lines follow, each of which contains two integers $$$t_i$$$, $$$v_i$$$, denoting the $$$i$$$-th query. If $$$t_i = 1$$$, then the $$$i$$$-th query is ADD $$$v_i$$$ ($$$0 \le v_i \le 29$$$). If $$$t_i = 2$$$, then the $$$i$$$-th query is GET $$$v_i$$$ ($$$0 \le v_i \le 10^9$$$). Output: For each GET query, print YES if it is possible to choose a subset with sum equal to $$$w$$$, or NO if it is impossible. Sample IO: Input: 5 1 0 1 0 1 0 2 3 2 4 Output: YES NO Input: 7 1 0 1 1 1 2 1 10 2 4 2 6 2 7 Output: YES YES YES

Title: Sheep Descrip

In [24]:
print(df.loc[311, "input_description"])
print()
print(df.loc[1111, "input_description"])
print()
print(df.loc[4567, "input_description"])
print()
print(df.loc[12345, "input_description"]) 

The first line contains a single integer $$$a~(1 \le a \le 10^{1000000})$$$.

The second line contains a single integer $$$l~(0 \le l \le 10^{1000000})$$$.

The third line contains a single integer $$$r~(0 \le r \le 10^{1000000})$$$.

It is guaranteed that $$$l \le r$$$.

It is also guaranteed that numbers $$$a, l, r$$$ contain no leading zeros.

The first input line contains one integer n (1 ≤ n ≤ 2000). Each of the following n lines contains two integers li and ri (1 ≤ li, ri ≤ 109; li ≤ ri).

The first line of the input contains an integer n (4 ≤ n ≤ 1000) — the number of straight sections of the track.

The following (n + 1)-th line contains pairs of integers (xi, yi) ( - 10 000 ≤ xi, yi ≤ 10 000). The first of these points is the starting position. The i-th straight section of the track begins at the point (xi, yi) and ends at the point (xi + 1, yi + 1).

It is guaranteed that:

- the first straight section is directed to the north;
- the southernmost (and if there are several, th

In [25]:
def find_constraints(inp):
    if not isinstance(inp, str):
        return 0.0

    text = unicodedata.normalize("NFKD", inp)
    text = re.sub(r"\\,\s*", "", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"(?<=\d)\s+(?=\d)", "", text)

    maxi = 0
    for b, e in re.findall(r"(\d+)\s*[⋅×*]\s*10\^\{(\d+)\}", text):
        maxi = max(maxi, int(b) * (10 ** int(e)))

    for b, e in re.findall(r"(\d+)\s*[⋅×*]\s*10\^(\d+)", text):
        maxi = max(maxi, int(b) * (10 ** int(e)))

    for e in re.findall(r"10\^\{(\d+)\}", text):
        maxi = max(maxi, 10 ** int(e))

    for e in re.findall(r"10\^(\d+)", text):
        maxi = max(maxi, 10 ** int(e))

    for b in re.findall(r"\b\d{1,}\b", text):
        maxi = max(maxi, int(b))

    return min(math.log10(maxi), 20) if maxi > 0 else 0.0


def find_math(text):
    if not isinstance(text, str) or len(text.strip()) == 0:
        return 0.0

    m1 = text.count('$')
    m2 = len(re.findall(r"\\[a-zA-Z]+", text))
    m3 = len(re.findall(r"[=<>+\-*/^_]", text))
    
    return m1+m2+m3


ALGO_KEYWORDS = [
    "dp", "dynamic programming", "graph", "tree", "dfs", "bfs",
    "shortest path", "dijkstra", "flow",
    "segment tree", "fenwick", "bit", "bits"
    "binary search", "two pointers", "sliding window",
    "greedy", "divide and conquer", "topological sort", 
    "mod", "modulo", "prime", "gcd", "lcm",
    "combinatorics", "permutation", "combination",
    "probability", "expected value",
    "matrix", "geometry", "logarithm"
]

def find_algowords(text, keywords):
    text = text.lower()
    return sum(1 for kw in keywords if kw in text)


In [26]:
tests = [
    ("he first line contains one integer $$$m$$$ ($$$1 \le m \le 10^5$$$) — the number of queries. Then $$$m$$$ lines follow, each of which contains two integers $$$t_i$$$, $$$v_i$$$, denoting the $$$i$$$-th query. If $$$t_i = 1$$$, then the $$$i$$$-th query is ADD $$$v_i$$$ ($$$0 \le v_i \le 29$$$). If $$$t_i = 2$$$, then the $$$i$$$-th query is GET $$$v_i$$$ ($$$0 \le v_i \le 10^9$$$).", 9),
    ("The first input line contains one integer n (1 ≤ n ≤ 2000). Each of the following n lines contains two integers li and ri (1 ≤ li, ri ≤ 109; li ≤ ri).", 3.3010),
    ("he first line of the input contains an integer n (4 ≤ n ≤ 1000) — the number of straight sections of the track. The following (n + 1)-th line contains pairs of integers (xi, yi) ( - 10 000 ≤ xi, yi ≤ 10 000). The first of these points is the starting position. The i-th straight section of the track begins at the point (xi, yi) and ends at the point (xi + 1, yi + 1). It is guaranteed that: - the first straight section is directed to the north; - the southernmost (and if there are several, then the most western of among them) point of the track is the first point; - the last point coincides with the first one (i.e., the start position); - any pair of straight sections of the track has no shared points (except for the neighboring ones, they share exactly one point); - no pair of points (except for the first and last one) is the same; - no two adjacent straight sections are directed in the same direction or in opposite directions.", 4),
    ("The first line of the input contains a positive integer, $T$ ($T \leq 200$), representing the number of test cases. Each of the next $T$ lines contains a single integer between $0$ and $4095$, inclusive, the target value for that test case.", 3.6123),
    ("The first line contains a single integer $$$a~(1 \le a \le 10^{1000000})$$$.", 20)
]

for text, expected in tests:
    out = find_constraints(text)
    print(f"Input: {text}")
    print(f"Output: {out:.4f}, Expected: {expected:.4f}")
    print()

Input: he first line contains one integer $$$m$$$ ($$$1 \le m \le 10^5$$$) — the number of queries. Then $$$m$$$ lines follow, each of which contains two integers $$$t_i$$$, $$$v_i$$$, denoting the $$$i$$$-th query. If $$$t_i = 1$$$, then the $$$i$$$-th query is ADD $$$v_i$$$ ($$$0 \le v_i \le 29$$$). If $$$t_i = 2$$$, then the $$$i$$$-th query is GET $$$v_i$$$ ($$$0 \le v_i \le 10^9$$$).
Output: 9.0000, Expected: 9.0000

Input: The first input line contains one integer n (1 ≤ n ≤ 2000). Each of the following n lines contains two integers li and ri (1 ≤ li, ri ≤ 109; li ≤ ri).
Output: 3.3010, Expected: 3.3010

Input: he first line of the input contains an integer n (4 ≤ n ≤ 1000) — the number of straight sections of the track. The following (n + 1)-th line contains pairs of integers (xi, yi) ( - 10 000 ≤ xi, yi ≤ 10 000). The first of these points is the starting position. The i-th straight section of the track begins at the point (xi, yi) and ends at the point (xi + 1, yi + 1). It is 

In [27]:
df["constraint"] = df["input_description"].apply(find_constraints)
df["math"] = df["combined_text"].apply(find_math)
df["algoword"] = df["combined_text"].apply( lambda x: find_algowords(x, ALGO_KEYWORDS))
df["length"] = df["description"].apply( lambda x: len(x) if isinstance(x, str) else 0 )

In [28]:
df[["constraint", "math", "algoword", "length"]].describe()

,constraint,math,algoword,length
count,13879.000000,13879.000000,13879.000000,13879.000000
mean,4.538912,94.541898,0.718279,993.062901
std,3.376594,104.041050,0.989077,534.328856
min,0.000000,0.000000,0.000000,0.000000
25%,2.025306,11.000000,0.000000,632.000000
50%,3.698970,46.000000,0.000000,906.000000
75%,5.698970,161.000000,1.000000,1262.000000
max,20.000000,955.000000,7.000000,7212.000000


In [29]:
df[["combined_text", "constraint", "math", "algoword", "length"]].sample(3, random_state=42)

df = df.drop(columns=["sample_io", "title", "description", "input_description", "output_description", "sample_io_text"])


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13879 entries, 0 to 13878
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   problem_class  13879 non-null  object 
 1   problem_score  13879 non-null  float64
 2   score_norm     13879 non-null  float64
 3   combined_text  13879 non-null  object 
 4   constraint     13879 non-null  float64
 5   math           13879 non-null  int64  
 6   algoword       13879 non-null  int64  
 7   length         13879 non-null  int64  
dtypes: float64(3), int64(3), object(2)
memory usage: 867.6+ KB


In [33]:
df.to_parquet("processed.parquet", index=False)

meta = {
  "feature_cols": ["combined_text", "constraint", "math", "algoword", "length"],
  "nrows": len(df)
}
with open("processed_meta.json", "w") as f:
    json.dump(meta, f)
